In [ ]:
#Prepare all our necessary libraries
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt

#pytorch libraries
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Pad, Compose,CenterCrop, ToPILImage, Normalize, ConvertImageDtype, Resize

from torchvision.models import resnet50
from torch import nn
from torch.nn import init, Linear, ReLU, Softmax
from torch.nn.init import xavier_uniform_
from torch.optim import SGD, Adam
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_
from seaborn import heatmap
import datetime

# libs für AE

!pip install cleverhans --upgrade
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("You are using Device: ", device)

In [ ]:
#Prepare our Dataset Structure, as it has to be normalized for pytorch

from torchvision.io import read_image

class GTSRB(Dataset):
    def __init__(self, annotations_file, img_dir , transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)[["Path","ClassId"]]
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        label = self.img_labels.iloc[idx, 1]
        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
img_dir = "/kaggle/input/gtsrb-german-traffic-sign/"
test_file = "/kaggle/input/gtsrb-german-traffic-sign/Test.csv"
test_data = GTSRB(img_dir = img_dir, annotations_file = test_file,
                  transform = Compose([Resize((30,30)), ConvertImageDtype(torch.float32)]))
from torch.utils.data import DataLoader
test_dataloader = DataLoader(test_data)


In [ ]:
# Display image and label.
test_features, test_labels = next(iter(test_dataloader))
print(f"Feature batch shape: {test_features.size()}")
print(f"Labels batch shape: {test_labels.size()}")
img = test_features[0]
label = test_labels[0]
img = ToPILImage()(img).convert("RGB")
plt.imshow(img)
plt.show()
print(f"Label: {label}")

In [ ]:
#downloading resent50 pretrained on ImageNet 


#adjust resnet50 to my dataset
class r50(nn.Module):
    def __init__(self, pretrained_model):
        super(r50,self).__init__()
        self.rn50 = pretrained_model
        self.fl1 = nn.Linear(1000, 256)
        self.fl2 = nn.Linear(256,43)
        
    def forward(self, X):
        X = self.rn50(X)
        X = F.relu(self.fl1(X))
        X = F.dropout(X, p=0.25)
        X = self.fl2(X)
        return X

In [ ]:
class conv_net(nn.Module):
    def __init__(self):
        super(conv_net,self).__init__()
        self.conv1 = nn.Conv2d(3,32,(5,5))
        self.conv2 = nn.Conv2d(32,64, (3,3))
        self.pool = nn.MaxPool2d((2,2))
        self.dropout1 = nn.Dropout(p=0.25)
        self.conv3 = nn.Conv2d(64,3,(3,3))
        self.linear1 = Linear(75,256)
        self.dropout2 = nn.Dropout(p=0.5)
        self.linear2 = Linear(256,43)

        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = self.pool(F.relu(self.conv2(X)))
        X = self.dropout1(X)
        X = self.pool(F.relu(self.conv3(X)))
        X = self.dropout1(X)
        X = torch.flatten(X,1)
        X = F.relu(self.linear1(X))
        X = self.dropout2(X)
        X = self.linear2(X)
        
        return X

SELECT YOUR MODEL HERE

In [ ]:
alternative_model = True

In [ ]:

if alternative_model:
    model = conv_net()
    PATH = '/kaggle/input/alternative-gtsrb/alt_gtsrb.pth'

else:
    resn50 = resnet50(pretrained=True, progress = True)
    model = r50(resn50)
    PATH = '/kaggle/input/gtsrb/gtsrbX.pth'

model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
model.eval()

In [ ]:
#put on cuda if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("You are using Device: ", device)
model.to(device)
test_features = test_features.to(device)
# Export the model for translation purposes
batch_size= 1
torch.onnx.export(model,                     # model being run
                  test_features,         # model input (or a tuple for multiple inputs)
                  "gtsrb.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

torch.save(test_data,"/kaggle/working/test_data.pt")
torch.save(test_data,"/kaggle/working/test_data.h5")
torch.save(test_data,"/kaggle/working/test_data.csv")

In [ ]:
#AE Generator 
#look at some AEs
ae_generator = ((projected_gradient_descent(model.to(device),image.to(device), 0.05, 0.0075, 10,np.inf,  clip_min=0, clip_max=1 ).detach().to(device), label) for image, label in test_dataloader)
for i in range(3):
    #print(next(ae_generator))
    ae = next(ae_generator)
    image = ToPILImage()(ae[0][0]).convert("RGB")
    plt.imshow(image)
    plt.show()
    print(ae[1])
    

In [ ]:
def heatmapping(cm):
    precision = []
    recall = []
    class_count = []
    for i in range(43):
        precision.append(round(cm[i][i]/cm[i].sum(),2))
        recall.append(round(cm[i][i]/cm[i].sum()))           
        class_count.append(cm[i].sum())
    metrics = pd.DataFrame({"precision" : precision, "recall" : recall, "class_count": class_count})
    
    plt.subplots(figsize=(20,15))
    heatmap(cm)
    return metrics

In [ ]:
cm = np.zeros((43,43))
predictions = []
truth = []
ae_count = len(test_dataloader)-10
epsilon = [0.2, 0.1,0.0925 ,0.085, 0.075, 0.065, 0.05, 0.025 ]
metrics = []
for eps in epsilon:
    correct = 0
    ae_generator = ((projected_gradient_descent(model.to(device),image.to(device),
                                                eps, eps/10, 15 ,np.inf,  clip_min=0, clip_max=1 ).detach().to(device), label) for image, label in test_dataloader)
    for i in range(ae_count):
        ae = next(ae_generator)
        image, label = ae[0].to(device), ae[1].to(device)

        predicted = F.softmax(model(image), dim = 1).argmax()

        predictions.append(predicted)
        truth.append(label)
        correct += (predicted == label)
        cm[int(label)][int(predicted)] += 1

    print('Accuracy of the network on the {tst_len} adversarial images with perturbation {e}: {acc}'.format(
        acc= int(correct) / ae_count, tst_len = ae_count, e = eps))
    metrics.append(heatmapping(cm))

In [ ]:
#check accuracy on test set
correct = 0
total = 0
cm = np.zeros((43,43))
predictions = []

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:        
        images, labels = data
        images, labels = images.to(device), labels.to(device)
            
        output = model(images)
        predicted = output.argmax()
        predictions.append(predicted)
        total += 1
        correct += (predicted == labels)
        cm[int(labels)][int(predicted)] += 1
        


print('Accuracy of the network on the {tst_len} test images: {acc}'.format(
    acc= int(correct) / int(total), tst_len=len(test_data)))


In [ ]:
f = open("/kaggle/working/metrics.csv", 'a')
for df in metrics:
    df.to_csv(f, decimal = ",")
f.close()

In [ ]:
"""precision: correctly predicted class A/predicted class A
 recall: number of correctly predicted class A photos out of the number of actual class A

precision = []
recall = []
class_count = []
for i in range(43):
    precision.append(round(cm[i][i]/cm[i].sum(),2))
    recall.append(round(cm[i][i]/cm[i].sum()))           
    class_count.append(cm[i].sum())
metrics = pd.DataFrame({"precision" : precision, "recall" : recall, "class_count": class_count})"""

In [ ]:
"""from seaborn import heatmap
plt.subplots(figsize=(20,15))
heatmap(cm)"""

In [ ]:
"""metrics"""